In [24]:
import tensorflow as tf
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Evaluator, Pusher
from tfx.orchestration import pipeline
import tensorflow_data_validation as tfdv
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from google.protobuf.json_format import MessageToDict
import pprint as pp
import os
from absl import logging

In [25]:
PIPELINE_NAME = "HAR-simple"
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)
#METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

logging.set_verbosity(logging.INFO)

In [26]:
DATA_ROOT = os.path.join('../data/root/')
context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

In [27]:
example_gen = CsvExampleGen(input_base=DATA_ROOT)
context.run(example_gen)

INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:Running executor for CsvExampleGen
INFO:absl:Generating examples.
INFO:absl:Processing input csv data ../data/root/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 34
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [28]:
artifact = example_gen.outputs['examples'].get()[0]
print(f'split names: {artifact.split_names}')
print(f'artifact uri: {artifact.uri}')

split names: ["train", "eval"]
artifact uri: pipelines/HAR-simple/CsvExampleGen/examples/34


In [29]:
train_uri = os.path.join(artifact.uri, 'Split-train')

tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

In [30]:
def get_records(dataset, num_records):
    '''Extracts records from the given dataset.
    Args:
        dataset (TFRecordDataset): dataset saved by ExampleGen
        num_records (int): number of records to preview
    SOURCE: Akila Somasundaram at https://medium.com/data-science/tensorflow-transform-ensuring-seamless-data-preparation-in-production-99ffcf49f535
    '''

    # initialize an empty list
    records = []

    # Use the `take()` method to specify how many records to get
    for tfrecord in dataset.take(num_records):

        # Get the numpy property of the tensor
        serialized_example = tfrecord.numpy()

        # Initialize a `tf.train.Example()` to read the serialized data
        example = tf.train.Example()

        # Read the example data (output is a protocol buffer message)
        example.ParseFromString(serialized_example)

        # convert the protocol bufffer message to a Python dictionary
        example_dict = (MessageToDict(example))

        # append to the records list
        records.append(example_dict)

    return records

In [31]:
sample_records = get_records(dataset, 3)

# Print the output
pp.pprint(sample_records)

[{'features': {'feature': {'activity_id': {'bytesList': {'value': ['c2l0dGluZw==']}},
                           'ax': {'floatList': {'value': [-0.001]}},
                           'ay': {'floatList': {'value': [0.849]}},
                           'az': {'floatList': {'value': [0.48433334]}},
                           'participant_id': {'bytesList': {'value': ['Wk0=']}},
                           'timestamp': {'floatList': {'value': [1760380200.0]}}}}},
 {'features': {'feature': {'activity_id': {'bytesList': {'value': ['c2l0dGluZw==']}},
                           'ax': {'floatList': {'value': [-0.008]}},
                           'ay': {'floatList': {'value': [0.85]}},
                           'az': {'floatList': {'value': [0.483]}},
                           'participant_id': {'bytesList': {'value': ['Wk0=']}},
                           'timestamp': {'floatList': {'value': [1760380200.0]}}}}},
 {'features': {'feature': {'activity_id': {'bytesList': {'value': ['c2l0dGluZw==']

2025-12-10 17:25:53.682221: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [32]:
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])
context.run(statistics_gen)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for StatisticsGen
INFO:absl:Generating statistics for split train.
INFO:absl:Statistics for split train written to pipelines/HAR-simple/StatisticsGen/statistics/35/Split-train.
INFO:absl:Generating statistics for split eval.
INFO:absl:Statistics for split eval written to pipelines/HAR-simple/StatisticsGen/statistics/35/Split-eval.
INFO:absl:Running publisher for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 35
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [33]:
context.show(statistics_gen.outputs['statistics'])

In [34]:
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    )
context.run(schema_gen)
context.show(schema_gen.outputs['schema'])

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for SchemaGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for SchemaGen
INFO:absl:Processing schema from statistics for split train.
INFO:absl:Processing schema from statistics for split eval.
INFO:absl:Schema written to pipelines/HAR-simple/SchemaGen/schema/36/schema.pbtxt.
INFO:absl:Running publisher for SchemaGen
INFO:absl:MetadataStore with DB connection initialized


,Type,Presence,Valency,Domain
Feature name,,,,
'activity_id',STRING,required,,'activity_id'
'ax',FLOAT,required,,-
'ay',FLOAT,required,,-
'az',FLOAT,required,,-
'participant_id',STRING,required,,'participant_id'
'timestamp',FLOAT,required,,-


,Values
Domain,
'activity_id',"'lying', 'running', 'sitting', 'walking'"
'participant_id',"'LL', 'VV', 'YL', 'YT', 'ZM'"


In [35]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])

context.run(example_validator)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for ExampleValidator
INFO:absl:Validating schema against the computed statistics for split train.
INFO:absl:Validation complete for split train. Anomalies written to pipelines/HAR-simple/ExampleValidator/anomalies/37/Split-train.
INFO:absl:Validating schema against the computed statistics for split eval.
INFO:absl:Validation complete for split eval. Anomalies written to pipelines/HAR-simple/ExampleValidator/anomalies/37/Split-eval.
INFO:absl:Running publisher for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: ExampleValidator
    execution_id: 37
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [36]:
context.show(example_validator.outputs['anomalies'])

Transform

In [37]:
CONSTANTS_MODULE_FILE = 'constants.py'

In [38]:
%%writefile {CONSTANTS_MODULE_FILE}

NUMERICAL_FEATURES = [
    'ax',
    'ay',
    'az', 
    'timestamp'
]

# BUCKET_FEATURES = [
#     ...
# ]
# # Number of buckets used by tf.transform for encoding each feature.
# FEATURE_BUCKET_COUNT = 10

CATEGORICAL_STRING_FEATURES = [
    'participant_id'
]

# Number of vocabulary terms used for encoding categorical features.
VOCAB_SIZE = 1000

# Count of out-of-vocab buckets in which unrecognized categorical are hashed.
# no bucket so set to 0
OOV_SIZE = 0

# Keys
LABEL_KEY = 'activity_id'

#rename feature keys to prevent clash with raw keys names
def t_name(key):
  return key + '_xf'

Overwriting constants.py


In [39]:
TRANSFORM_MODULE_FILE = 'transform.py'

In [41]:
%%writefile {TRANSFORM_MODULE_FILE}

import tensorflow as tf
import tensorflow_transform as tft
# constant are saved in cached so for updated changes, reload imports and file
import constants
import sys
if 'google.colab' in sys.modules:  # Testing to see if we're doing development
  import importlib
  importlib.reload(constants)

_NUMERICAL_FEATURES = constants.NUMERICAL_FEATURES
# _BUCKET_FEATURES = constants.BUCKET_FEATURES
# _FEATURE_BUCKET_COUNT = constants.FEATURE_BUCKET_COUNT
# _CATEGORICAL_NUMERICAL_FEATURES = constants.CATEGORICAL_NUMERICAL_FEATURES
_CATEGORICAL_STRING_FEATURES = constants.CATEGORICAL_STRING_FEATURES
_VOCAB_SIZE = constants.VOCAB_SIZE
_OOV_SIZE = constants.OOV_SIZE
_LABEL_KEY = constants.LABEL_KEY

# one-hot tensor to encode categorical features
# returns dense one hot tensor as flost list
# x is dense tensor
# key is string key for feature input 
def make_one_hot(x, key):
  integerized = tft.compute_and_apply_vocabulary(x,
          top_k=_VOCAB_SIZE,
          num_oov_buckets=_OOV_SIZE,
          vocab_filename=key, name=key)
  depth = (
      tft.experimental.get_vocabulary_size_by_name(key) + _OOV_SIZE)
  one_hot_encoded = tf.one_hot(
      integerized,
      depth=tf.cast(depth, tf.int32),
      on_value=1.0,
      off_value=0.0)
  return tf.reshape(one_hot_encoded, [-1, depth])

# fill missing vals of x with ''/0
def fill_in_missing(x):
  if not isinstance(x, tf.sparse.SparseTensor):
    return x

  default_value = '' if x.dtype == tf.string else 0
  return tf.squeeze(
      tf.sparse.to_dense(
          tf.SparseTensor(x.indices, x.values, [x.dense_shape[0], 1]),
          default_value),
      axis=1)


# preprocessing inputs
def preprocessing_fn(inputs):
  outputs = {}
  for key in _NUMERICAL_FEATURES:
    # If sparse make it dense, setting nan's to 0 or '', and apply zscore.
    outputs[constants.t_name(key)] = tft.scale_to_z_score(
        fill_in_missing(inputs[key]), name=key)

  #   for key in _BUCKET_FEATURES:
  #     outputs[constants.t_name(key)] = tf.cast(tft.bucketize(
  #             fill_in_missing(inputs[key]), _FEATURE_BUCKET_COUNT, name=key),
  #             dtype=tf.float32)

    for key in _CATEGORICAL_STRING_FEATURES:
      outputs[constants.t_name(key)] = make_one_hot(fill_in_missing(inputs[key]), key)

  #   for key in _CATEGORICAL_NUMERICAL_FEATURES:
  #     outputs[constants.t_name(key)] = make_one_hot(tf.strings.strip(
  #         tf.strings.as_string(fill_in_missing(inputs[key]))), key)


  # Convert label to int
  # 0: lying, 1: running, 2: sitting, 3: walking
  outputs[constants.LABEL_KEY] = tft.compute_and_apply_vocabulary(
      fill_in_missing(inputs[constants.LABEL_KEY]),
      top_k=None,                 # keep all classes
      num_oov_buckets=0,          # no out-of-vocab
      vocab_filename=constants.LABEL_KEY
  )
  # returns feature to transformed feature operations
  return outputs

Overwriting transform.py


In [42]:
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(TRANSFORM_MODULE_FILE))
context.run(transform, enable_cache=True)

INFO:absl:Generating ephemeral wheel package for '/Users/laura/VSCodeProj/ML_project/tfx/transform.py' (including modules: ['constants', 'transform']).
INFO:absl:User module package has hash fingerprint version b8ed1f62d6023703c51211b1a7c8f459c0ef1208807612e20a6bb413a2001255.
INFO:absl:Executing: ['/Users/laura/VSCodeProj/ML_project/.venv/bin/python', '/var/folders/cx/76gdbss126gcsh3vj3352hlr0000gn/T/tmpfyiz5m52/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/var/folders/cx/76gdbss126gcsh3vj3352hlr0000gn/T/tmpxxny3t9o', '--dist-dir', '/var/folders/cx/76gdbss126gcsh3vj3352hlr0000gn/T/tmplq76gv82']
INFO:absl:Successfully built user code wheel distribution at 'pipelines/HAR-simple/_wheels/tfx_user_code_Transform-0.0+b8ed1f62d6023703c51211b1a7c8f459c0ef1208807612e20a6bb413a2001255-py3-none-any.whl'; target user module is 'transform'.
INFO:absl:Full user module path is 'transform@pipelines/HAR-simple/_wheels/tfx_user_code_Transform-0.0+b8ed1f62d6023703c51211b1a7c8f459c0ef120880761

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying constants.py -> build/lib
copying transform.py -> build/lib
installing to /var/folders/cx/76gdbss126gcsh3vj3352hlr0000gn/T/tmpxxny3t9o
running install
running install_lib
copying build/lib/constants.py -> /var/folders/cx/76gdbss126gcsh3vj3352hlr0000gn/T/tmpxxny3t9o
copying build/lib/transform.py -> /var/folders/cx/76gdbss126gcsh3vj3352hlr0000gn/T/tmpxxny3t9o
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /var/

INFO:absl:Successfully installed 'pipelines/HAR-simple/_wheels/tfx_user_code_Transform-0.0+b8ed1f62d6023703c51211b1a7c8f459c0ef1208807612e20a6bb413a2001255-py3-none-any.whl'.
INFO:absl:udf_utils.get_fn {'module_file': None, 'module_path': 'transform@pipelines/HAR-simple/_wheels/tfx_user_code_Transform-0.0+b8ed1f62d6023703c51211b1a7c8f459c0ef1208807612e20a6bb413a2001255-py3-none-any.whl', 'stats_options_updater_fn': None} 'stats_options_updater_fn'
INFO:absl:Installing 'pipelines/HAR-simple/_wheels/tfx_user_code_Transform-0.0+b8ed1f62d6023703c51211b1a7c8f459c0ef1208807612e20a6bb413a2001255-py3-none-any.whl' to a temporary directory.
INFO:absl:Executing: ['/Users/laura/VSCodeProj/ML_project/.venv/bin/python', '-m', 'pip', 'install', '--target', '/var/folders/cx/76gdbss126gcsh3vj3352hlr0000gn/T/tmp1z0euvqn', 'pipelines/HAR-simple/_wheels/tfx_user_code_Transform-0.0+b8ed1f62d6023703c51211b1a7c8f459c0ef1208807612e20a6bb413a2001255-py3-none-any.whl']


Processing ./pipelines/HAR-simple/_wheels/tfx_user_code_Transform-0.0+b8ed1f62d6023703c51211b1a7c8f459c0ef1208807612e20a6bb413a2001255-py3-none-any.whl


INFO:absl:Successfully installed 'pipelines/HAR-simple/_wheels/tfx_user_code_Transform-0.0+b8ed1f62d6023703c51211b1a7c8f459c0ef1208807612e20a6bb413a2001255-py3-none-any.whl'.
INFO:absl:Installing 'pipelines/HAR-simple/_wheels/tfx_user_code_Transform-0.0+b8ed1f62d6023703c51211b1a7c8f459c0ef1208807612e20a6bb413a2001255-py3-none-any.whl' to a temporary directory.
INFO:absl:Executing: ['/Users/laura/VSCodeProj/ML_project/.venv/bin/python', '-m', 'pip', 'install', '--target', '/var/folders/cx/76gdbss126gcsh3vj3352hlr0000gn/T/tmpxqphf5qc', 'pipelines/HAR-simple/_wheels/tfx_user_code_Transform-0.0+b8ed1f62d6023703c51211b1a7c8f459c0ef1208807612e20a6bb413a2001255-py3-none-any.whl']


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.
INFO:absl:Running publisher for Transform
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Transform
    execution_id: 38
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [43]:
transform.outputs

{'transform_graph': OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False),
 'transformed_examples': OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False),
 'updated_analyzer_cache': OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False),
 'pre_transform_schema': OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False),
 'pre_transform_stats': OutputChannel(artifact_type=ExampleStatistics, producer_componen

In [44]:
train_uri = transform.outputs['transform_graph'].get()[0].uri
os.listdir(train_uri)

['transformed_metadata', 'transform_fn', 'metadata']

In [45]:
# Get the URI of the output artifact representing the transformed examples, which is a directory
train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and decode them.
for tfrecord in dataset.take(3):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

features {
  feature {
    key: "activity_id"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "ax_xf"
    value {
      float_list {
        value: -0.2709307372570038
      }
    }
  }
  feature {
    key: "ay_xf"
    value {
      float_list {
        value: 1.4972364902496338
      }
    }
  }
  feature {
    key: "az_xf"
    value {
      float_list {
        value: 0.5400288701057434
      }
    }
  }
  feature {
    key: "participant_id_xf"
    value {
      float_list {
        value: 0.0
        value: 0.0
        value: 1.0
        value: 0.0
        value: 0.0
      }
    }
  }
  feature {
    key: "timestamp_xf"
    value {
      float_list {
        value: -1.5508753061294556
      }
    }
  }
}

features {
  feature {
    key: "activity_id"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "ax_xf"
    value {
      float_list {
        value: -0.28597861528396606
      }
    }
  }
  feature {
    k

2025-12-10 17:45:28.719414: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Trainer

In [22]:
TRAINER_MODULE_FILE = 'trainer.py'